In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
processor = TrOCRProcessor.from_pretrained("raxtemur/trocr-base-ru")  # русская версия
model = VisionEncoderDecoderModel.from_pretrained("raxtemur/trocr-base-ru")



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
from PIL import Image
image = Image.open("dataset/receipt.jpg")
pixel_values = processor(images=image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)
text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [4]:
text

'УЕФА'

In [5]:
import cv2
import pytesseract
from pytesseract import Output

img = cv2.imread('dataset/receipt.jpg', cv2.IMREAD_GRAYSCALE)
# Предобработка: порог, шумоподавление
th = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)[1]
d = pytesseract.image_to_data(th, output_type=Output.DICT)
items = []
for i, word in enumerate(d['text']):
    text = word.strip()
    if text and any(c.isdigit() for c in text):
        x,y,w,h = d['left'][i], d['top'][i], d['width'][i], d['height'][i]
        items.append({'word': text, 'bbox': (x,y,w,h)})
# Постобработка: группировка слов в строки и поиск "название — цена"


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [ ]:
import requests

API_KEY = 'helloworld'  # бесплатный ключ OCR.space
def ocr_receipt(path):
    with open(path, 'rb') as f:
        r = requests.post(
            'https://api.ocr.space/parse/image',
            files={'file': f},
            data={'apikey': API_KEY, 
                  'language': 'rus', 
                  'OCREngine': 1,           # обязательный параметр для новых языков
                  'isOverlayRequired': True}
        )
    return r.json()

data = ocr_receipt('dataset/receipt.jpg')
for entry in data['ParsedResults'][0]['TextOverlay']['Lines']:
    print(entry['LineText'], entry['Words'])


KeyboardInterrupt: 

: 

In [15]:
data

{'OCRExitCode': 3,
 'IsErroredOnProcessing': True,
 'ErrorMessage': ['File failed validation. File size exceeds the maximum permissible file size limit of 1024 KB'],
 'ErrorDetails': '',
 'ProcessingTimeInMilliseconds': '15'}

In [10]:
data['ParsedResults'][0]['TextOverlay']['Lines']

[{'LineText': '000"nponropr"',
  'Words': [{'WordText': '000"nponropr"',
    'Left': 429.0,
    'Top': 113.0,
    'Height': 19.0,
    'Width': 136.0}],
  'MaxHeight': 19.0,
  'MinTop': 113.0},
 {'LineText': '690033. r 8naagoocT0K,np-KT',
  'Words': [{'WordText': '690033.',
    'Left': 270.0,
    'Top': 121.0,
    'Height': 18.0,
    'Width': 55.0},
   {'WordText': 'r',
    'Left': 330.0,
    'Top': 129.0,
    'Height': 10.0,
    'Width': 6.0},
   {'WordText': '8naagoocT0K,np-KT',
    'Left': 340.0,
    'Top': 128.0,
    'Height': 24.0,
    'Width': 146.0}],
  'MaxHeight': 24.0,
  'MinTop': 121.0},
 {'LineText': 'KACCOBbtg 142',
  'Words': [{'WordText': 'KACCOBbtg',
    'Left': 371.0,
    'Top': 154.0,
    'Height': 18.0,
    'Width': 65.0},
   {'WordText': '142',
    'Left': 478.0,
    'Top': 159.0,
    'Height': 12.0,
    'Width': 24.0}],
  'MaxHeight': 18.0,
  'MinTop': 154.0},
 {'LineText': '00109511"',
  'Words': [{'WordText': '00109511"',
    'Left': 250.0,
    'Top': 185.0,
    '

In [7]:
import vk_api
from vk_api.bot_longpoll import VkBotLongPoll, VkBotEventType
import requests
import os
# Инициализация с использованием токена сообщества
VK_TOKEN = os.getenv("vk_api")
vk_session = vk_api.VkApi(token=VK_TOKEN)
longpoll = VkBotLongPoll(vk_session, group_id=231492060)  # укажите ID группы числом
vk = vk_session.get_api()  # объект для вызова методов VK API

# Хранилище данных пользователей
user_data = {}  # словарь {user_id: {"name": ..., "phone": ..., "bank": ...}}
user_state = {}  # словарь для отслеживания этапа регистрации {user_id: "awaiting_name"/"awaiting_phone"/...}


In [8]:
def handle_personal_message(user_id, message_text):
    """Обработка личного сообщения от пользователя (регистрация)."""
    # Если пользователь еще не начал регистрацию, либо явно ввел команду "/start"
    if user_id not in user_data or message_text.lower() == "/start":
        user_state[user_id] = "awaiting_name"
        vk.messages.send(
            user_id=user_id,
            message="Здравствуйте! Для регистрации в системе разделения счетов введите, пожалуйста, свое ФИО.",
            random_id=0
        )
        return

    # Если ожидаем ввод ФИО
    if user_state.get(user_id) == "awaiting_name":
        # Сохраняем ФИО и переходим к следующему шагу
        full_name = message_text.strip()
        user_data[user_id] = {"name": full_name}
        user_state[user_id] = "awaiting_phone"
        vk.messages.send(
            user_id=user_id,
            message=f"Спасибо, {full_name}! Теперь отправьте номер телефона, привязанный к вашему банковскому аккаунту (для переводов).",
            random_id=0
        )
        return

    # Если ожидаем ввод телефона
    if user_state.get(user_id) == "awaiting_phone":
        phone = message_text.strip()
        user_data[user_id]["phone"] = phone
        user_state[user_id] = "awaiting_bank"
        vk.messages.send(
            user_id=user_id,
            message="Отлично. Укажите предпочтительный банк для переводов (например, Сбербанк, Тинькофф и т.д.).",
            random_id=0
        )
        return

    # Если ожидаем ввод банка
    if user_state.get(user_id) == "awaiting_bank":
        bank = message_text.strip()
        user_data[user_id]["bank"] = bank
        user_state[user_id] = None  # регистрация завершена
        vk.messages.send(
            user_id=user_id,
            message=f"Вы успешно зарегистрированы! ФИО: {user_data[user_id]['name']}, Телефон: {user_data[user_id]['phone']}, Банк: {user_data[user_id]['bank']}.",
            random_id=0
        )
        # Опционально: уведомить, что теперь можно добавлять бота в беседы
        vk.messages.send(
            user_id=user_id,
            message="Теперь вы можете добавить бота в беседу для разделения счетов.",
            random_id=0
        )
        return


In [ ]:
"""Пример минимального бота ВКонтакте, который отвечает пользователям
личным сообщением. Подготовлен для python-vk_api >= 11.9.
"""
import os
import vk_api
from vk_api.bot_longpoll import VkBotLongPoll, VkBotEventType
from vk_api.utils import get_random_id

VK_TOKEN = os.getenv("vk_api")  # Токен сообщества в переменных окружения
GROUP_ID = 231492060  # ID вашего сообщества

# Инициализируем сессию и объекты API
vk_session = vk_api.VkApi(token=VK_TOKEN)
longpoll = VkBotLongPoll(vk_session, group_id=GROUP_ID)
vk = vk_session.get_api()

# Простейшие хранилища, если понадобится сохранять данные о пользователях
user_data = {}
user_state = {}

def send_private_message(user_id: int, text: str) -> None:
    """Отправляет личное сообщение пользователю от имени сообщества."""
    vk.messages.send(
        peer_id=user_id,             # Для личного сообщения достаточно ID пользователя
        random_id=get_random_id(),   # Уникальный random_id предотвращает дубли
        message=text
    )

def main() -> None:
    print("Bot started — listening for events…")
    for event in longpoll.listen():
        # Отслеживаем входящие личные сообщения пользователям
        if event.type == VkBotEventType.MESSAGE_NEW and event.from_user:
            user_id = event.message["from_id"]
            text = event.message.get("text", "").strip().lower()

            if text == "/start":
                send_private_message(user_id, "Привет! Я бот и теперь могу писать в личку 😉")
            else:
                send_private_message(user_id, f"Вы написали: {text or '...' }")

if __name__ == "__main__":
    main()


Bot started — listening for events…


In [ ]:
vk.messages.send(
    peer_id=user_id,          # личка
    random_id=get_random_id(),
    message="123"
)

In [ ]:
""